In [ ]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
from readparquet import UBParquetReader
from detectoroutline import DetectorOutline

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255),  # proton
                               (0,0,0)),     # other
                               dtype=np.float ) 

larcvpid2class = {0:0,#unknown -> bg
                  1:0,#cosmic -> bg
                  2:0,#bnb -> bg
                  3:1,#eminus->electron
                  4:2,#gamma->gamma
                  5:2,#pizero->gamma
                  6:3,#muon->muon
                  7:4,#kaon->other
                  8:4,#pion->pion
                  9:5,#proton->proton
                 }

In [ ]:
# DATA FILES

# specify location where parquet file(s) live
datafolder="./instance_data/"

reader = UBParquetReader(datafolder)
NENTRIES = reader.nentries

In [ ]:
# Get entry data
ENTRY = 101
data = reader.get_entry(ENTRY)
print("data keys: ",data.keys())
print("nvoxels: ",data["instvoxcoord"].shape[0])

In [ ]:
# Print Entry Info
#for col in ['run','subrun','event','truepos','nu_energy','nu_pid','nu_ccnc','nu_interaction','nu_geniemode']:
#    print(col,": ",data[col])

# make instance 2 id map
#instance2id = {}
#for k,i in zip(data['voxinstancelist'],data['voxidlist']):
#    instance2id[k.as_py()] = i.as_py()
#for primdata in zip(data['primary_trackid'].as_py(),data['primary_pid'].as_py(),data['primary_start'].as_py(),data['primary_mom'].as_py()):
#    trackid=primdata[0]
#    if trackid in instance2id:
#        print('[%d]'%(instance2id[primdata[0]]),": ",primdata)
    
#print(instance2id)

In [ ]:
# GENERATE PLOTLY OBJECTS FOR VISUALIZATION
plotdata = []
nvoxels = data["instvoxcoord"].shape[0]
# We need to retrieved the 3d positions
pos3d = data["instvoxcoord"].astype(np.float)*1.0
pos3d[:,1] -= 117.0 


In [ ]:
# Plot Particle
no_ghost_points = True
pos = pos3d
label = data["label"].as_py()
labelcol = ssnet_color_array[label]


instance_plot = {
    "type":"scatter3d",
    "x":pos[:,0],
    "y":pos[:,1],
    "z":pos[:,2],
        "mode":"markers",
        "name":"ssnet",
        "marker":{"color":labelcol,"size":1}
    }

detlines = detdata.getlines(color=(10,10,10))

# DATA
instance_plot_data = [instance_plot] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='SINGLE PARTICLE VOXELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 4.0, "y": 0.5, "z": -0.5},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=instance_plot_data, layout=layout)
fig.show()